#### This program will build a more advanced and refined Brochure utilizing Multi-shot prompting

Import Statements

In [62]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

Load OpenAI API Key

In [63]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


WebPage Class

In [64]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

Link System Prompt

In [65]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [66]:
# print(link_system_prompt)

Function to get links based on User Prompt
- Purpose: Constructs a prompt for extracting relevant links.
- Input: website (a Website object containing the URL and links).
- Steps:
    1. Combines the URL and links into a prompt.
    2. Instructs the model to exclude irrelevant links (e.g., Terms of Service).
- Output: A structured prompt for link extraction.

In [67]:
# def get_links_user_prompt(website):
#     user_prompt = f"Here is the list of links on the website of {website.url} - "
#     user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
# Do not include Terms of Service, Privacy, email links.\n"
#     user_prompt += "Links (some might be relative links):\n"
#     user_prompt += "\n".join(website.links)
#     return user_prompt

Get Links Function
- Purpose: Extracts and categorizes relevant links from the webpage.
- Input: url (webpage URL).
- Steps:
    1. Initializes a Website object with the given URL.
    2. Constructs a system prompt (link_system_prompt) and a user prompt via get_links_user_prompt.
    3. Calls OpenAI's API with the prompts to categorize the links.
    4. Parses and returns the API's JSON response.
- Output: A dictionary of categorized links (e.g., About page, Careers page).

In [68]:
# def get_links(url):
#     website = Website(url)
#     response = openai.chat.completions.create(
#         model=MODEL,
#         messages=[
#             {"role": "system", "content": link_system_prompt},
#             {"role": "user", "content": get_links_user_prompt(website)}
#       ],
#         response_format={"type": "json_object"}
#     )
#     result = response.choices[0].message.content
#     return json.loads(result)

#  Expanding on original work for Multi Shot prompting

Fetch All Links Functions

In [69]:
def fetch_all_links(website):
    user_prompt = f"Here is the list of links found on the website {website.url}:\n"
    user_prompt += "\n".join(website.links)
    user_prompt += "\nAnalyze these links to extract the most relevant ones for a company brochure."
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": user_prompt}
        ],
    )
    return response.choices[0].message.content


Categorize Links Function

In [70]:
def categorize_links(links):
    user_prompt = f"The following links were extracted:\n{links}\n"
    user_prompt += "Categorize each link into one of the following types: 'About Page,' 'Careers Page,' 'Contact Page,' or 'Other.'"
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": user_prompt}
        ],
    )
    return response.choices[0].message.content


Filter Relevant Links Function

In [71]:
def filter_relevant_links(categorized_links):
    user_prompt = f"The following links have been categorized:\n{categorized_links}\n"
    user_prompt += "Filter out irrelevant links (e.g., 'Terms of Service,' 'Privacy Policy'). Return only the relevant links in JSON format."
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": user_prompt}
        ],
    )
    return response.choices[0].message.content


Confirm Links Function

In [72]:
def confirm_links(filtered_links):
    user_prompt = f"The following links have been filtered and deemed relevant:\n{filtered_links}\n"
    user_prompt += "Please confirm these are the best links for a company brochure."
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": user_prompt}
        ],
    )
    return response.choices[0].message.content


Get Links Multi Shot Function

In [73]:
def get_links_multi_shot(url):
    website = Website(url)
    
    # Step 1: Fetch all links
    all_links = fetch_all_links(website)
    print("All Links Fetched:", all_links)
    
    # Step 2: Categorize links
    categorized_links = categorize_links(all_links)
    print("Categorized Links:", categorized_links)
    
    # Step 3: Filter relevant links
    relevant_links = filter_relevant_links(categorized_links)
    print("Filtered Relevant Links:", relevant_links)
    
    # Step 4: Confirm final links
    final_links = confirm_links(relevant_links)
    print("Final Links:", final_links)
    
    return final_links


Get All Details function
- Purpose: Fetches detailed content from the landing page and relevant links.
- Input: url (landing page URL).
- Steps:
    1. Fetches the landing page content using Website(url).get_contents().
    2. Calls get_links(url) to retrieve and categorize relevant links.
    3. Iterates over the links, fetches their content, and appends it to the result.
- Output: Combined content of the landing page and relevant links.

In [74]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links_multi_shot(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

# We pass in a URL
# the function goes to the landing page
# Then lists the contents of the landing page
# Then calls the previously made function calling gpt 4-0 mini
# Prints that the links are found
# Then goes through the list of links and performs the same process again

Assistant System Prompt

In [75]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

Get Brochure User Prompt Function
- Purpose: Constructs the user-specific prompt for the GPT model.
- Input: company_name and url.
- Steps:
    1. Calls get_all_details(url) to retrieve the content of the landing page and relevant links.
    2. Constructs a prompt combining the company name and the content retrieved from get_all_details.
    3. Truncates the prompt to 5,000 characters if necessary.
- Output: A detailed user prompt for generating the brochure.

In [76]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [77]:
# get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Create Brochure Function
- Generates the final Brochure using OpenAI's GPT Model
- input: company_name and url
- Steps: 
    1. Calls get_brochure_user_prompt to generate a detailed user prompt.
    2. Sends the system and user prompts to OpenAI's API.
    3. Displays the markdown output using Markdown()

In [78]:
# def create_brochure(company_name, url):
#     response = openai.chat.completions.create(
#         model=MODEL,
#         messages=[
#             {"role": "system", "content": system_prompt},
#             {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
#           ],
#     )
#     result = response.choices[0].message.content
#     display(Markdown(result))

Multi-shot Create Brochure Function

In [79]:
def create_brochure(company_name, url):
    # Step 1: Generate About section
    about_prompt = f"Write a concise and professional 'About Us' section for {company_name} based on information from {url}."
    about_response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": about_prompt}
        ],
    )
    about_section = about_response.choices[0].message.content

    # Step 2: Generate Careers section
    careers_prompt = f"Write an engaging 'Careers' section for {company_name}, highlighting job opportunities and company culture."
    careers_response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": careers_prompt}
        ],
    )
    careers_section = careers_response.choices[0].message.content

    # Step 3: Generate Contact Information section
    contact_prompt = f"Write a 'Contact Information' section for {company_name} based on information from {url}."
    contact_response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": contact_prompt}
        ],
    )
    contact_section = contact_response.choices[0].message.content

    # Combine all sections into a single brochure
    brochure = f"# {company_name} Brochure\n\n"
    brochure += f"## \n{about_section}\n\n"
    brochure += f"## \n{careers_section}\n\n"
    brochure += f"## \n{contact_section}\n\n"

    # Display the final brochure
    display(Markdown(brochure))


In [80]:
create_brochure("HuggingFace", "https://huggingface.com")

# HuggingFace Brochure

## 
# About Us

At Hugging Face, we are transforming the landscape of artificial intelligence by providing innovative natural language processing (NLP) solutions that are accessible and efficient. Founded in 2016, our mission is to democratize AI and make it available to everyone. We believe that state-of-the-art tools should not be limited to large organizations but should empower individuals and small teams to experiment, innovate, and create.

Our core product, the Hugging Face Transformers library, offers a range of pre-trained models that facilitate the development and deployment of machine learning applications. From text classification to question answering and beyond, our library has become the go-to resource for researchers, developers, and AI enthusiasts.

We pride ourselves on our open-source philosophy, fostering a vibrant community around machine learning, and collaborating with researchers and engineers across the globe. Our commitment to transparency and accessibility encourages knowledge sharing and collaboration, making it easier for everyone to contribute to the AI revolution.

At Hugging Face, we champion a diverse and inclusive company culture that values creativity, curiosity, and collaboration. Our team is composed of talented individuals from various backgrounds who are passionate about technology and its potential to change lives. We offer opportunities for growth and development, ensuring that every team member can thrive.

Join us in our journey to make AI technology human-friendly and truly transformative for society. Whether you are a prospective customer, investor, or future team member, we invite you to explore the endless possibilities with Hugging Face.

## 
# Careers at HuggingFace

## Join Us in Shaping the Future of AI

At HuggingFace, we are on a mission to democratize artificial intelligence and make it accessible to everyone. Our collaborative team is made up of passionate innovators from diverse backgrounds who thrive on tackling complex challenges. We believe that great ideas come from a mix of experiences, and we encourage open dialogue and a culture of experimentation. 

### Why Work at HuggingFace?

- **Impactful Projects**: Work on groundbreaking technologies and projects that pave the way for the future of AI and machine learning. Your contributions will have a direct impact on developers, researchers, and everyday users worldwide.
  
- **Inclusive Culture**: Our culture is built on inclusion and belonging. We celebrate diversity and are committed to creating a workspace where every voice is heard and valued. We believe in equal opportunity for all and strive for representation in our workforce.
  
- **Continuous Learning**: At HuggingFace, we support professional growth through mentorship, workshops, and access to educational resources. Whether you're an experienced professional or just starting, you’ll have the tools you need to expand your expertise.
  
- **Collaborative Environment**: Collaboration and teamwork are at the heart of what we do. Our flat organization encourages open communication, where everyone feels empowered to share their ideas and feedback, fostering a dynamic exchange of knowledge among colleagues.

### Opportunities

We are looking for passionate individuals across various fields, including:

- **Machine Learning Engineers**: If you are an expert in natural language processing or deep learning, you’ll find exciting opportunities to design and implement cutting-edge models.

- **Data Scientists**: Join us in deriving insights and building predictive models that help shape the way our technologies function and evolve.

- **Frontend and Backend Developers**: Contribute to our user-friendly platforms, ensuring seamless interaction for our community of users.

- **Product Managers**: Help us innovate and focus on delivering products that meet the needs of our users while keeping us aligned with our mission.

- **Marketing and Community Engagement**: Play a vital role in building our brand and connecting with the global AI community, sharing knowledge and resources.

### Benefits

- **Flexible Work Arrangements**: Enjoy a healthy work-life balance with remote work options and flexible hours that fit your lifestyle.

- **Health and Wellness**: We prioritize the well-being of our team members, providing comprehensive health benefits and wellness programs to support a healthy lifestyle.

- **Team Retreats and Social Events**: Engage with your colleagues beyond work with team-building retreats, hackathons, and casual gatherings to build camaraderie.

### Get Involved!

Are you excited to make a difference in the world of artificial intelligence? If you’re passionate, innovative, and ready to take the leap, we want to meet you! Explore our current job openings on our [careers page](#) and submit your application today.

At HuggingFace, your journey begins here. Together, we can push the boundaries of what's possible in AI. Join us and be a part of something revolutionary!

## 
# Contact Information

For inquiries, support, or any assistance, please reach out to Hugging Face through the following channels:

- **Website:** [huggingface.com](https://huggingface.com)
- **Email:** contact@huggingface.co
- **Social Media:**
  - Twitter: [@huggingface](https://twitter.com/huggingface)
  - LinkedIn: [Hugging Face](https://www.linkedin.com/company/huggingface)
  - GitHub: [Hugging Face GitHub](https://github.com/huggingface)
  
Whether you’re a customer looking for support, an investor interested in our growth, or a prospective recruit eager to join our team, we would love to hear from you!



Translation to Spanish Function

In [90]:
def translate_brochure_to_spanish(brochure_text):
    # Prompt to translate the brochure
    translation_to_spanish_prompt = (
        f"Translate the following text into Spanish:\n\n{brochure_text}"
    )
    # Call the LLM for translation
    response = openai.chat.completions.create(
        model=MODEL,  # Ensure the model supports multilingual tasks
        messages=[
            {"role": "system", "content": "You are a professional translator."},
            {"role": "user", "content": translation_to_spanish_prompt}
        ],
    )
    # Extract and return the translated content
    translated_text = response.choices[0].message.content
    return translated_text


In [91]:
def create_brochure_in_spanish(company_name, url, translate=False):
    # Step 1: Generate About section
    about_prompt = f"Write a concise and professional 'About Us' section for {company_name} based on information from {url}."
    about_response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": about_prompt}
        ],
    )
    about_section = about_response.choices[0].message.content

    # Step 2: Generate Careers section
    careers_prompt = f"Write an engaging 'Careers' section for {company_name}, highlighting job opportunities and company culture."
    careers_response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": careers_prompt}
        ],
    )
    careers_section = careers_response.choices[0].message.content

    # Step 3: Generate Contact Information section
    contact_prompt = f"Write a 'Contact Information' section for {company_name} based on information from {url}."
    contact_response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": contact_prompt}
        ],
    )
    contact_section = contact_response.choices[0].message.content

    # Combine all sections into a single brochure
    brochure = f"# {company_name} Brochure\n\n"
    brochure += f"## \n{about_section}\n\n"
    brochure += f"## \n{careers_section}\n\n"
    brochure += f"## \n{contact_section}\n\n"

    if translate:
        brochure = translate_brochure_to_spanish(brochure)

    # Display the final brochure
    display(Markdown(brochure))


In [92]:
create_brochure_in_spanish("HuggingFace", "https://huggingface.com", True)

# Folleto de Hugging Face

## 
# Sobre Nosotros

¡Bienvenido a Hugging Face!

En Hugging Face, estamos comprometidos con la democratización de la inteligencia artificial. Fundada en 2016, hemos crecido rápidamente hasta convertirnos en una organización líder en investigación de IA y plataforma de código abierto especializada en Procesamiento de Lenguaje Natural (NLP) y aprendizaje profundo. Nuestra misión es hacer que la tecnología de IA sea accesible y reutilizable para todos: investigadores, desarrolladores y empresas por igual.

Nuestra vibrante comunidad se basa en una cultura de colaboración e inclusividad, donde la innovación florece. Creemos en el poder de las contribuciones de código abierto y alentamos activamente a nuestros usuarios a compartir y colaborar en proyectos, fomentando un rico ecosistema de conocimiento y recursos compartidos.

Estamos orgullosos de apoyar a una amplia gama de clientes, desde startups hasta grandes empresas, ofreciendo herramientas y modelos de vanguardia que simplifican y agilizan la integración de IA. Nuestros productos insignia, como la biblioteca Transformers y el Hugging Face Hub, empoderan a los usuarios para construir, compartir y desplegar sus propios modelos de aprendizaje automático de última generación, transformando la forma en que las empresas aprovechan la IA.

Únete a nosotros en nuestro viaje para dar forma al futuro de la IA. Siempre estamos en busca de personas talentosas que compartan nuestra pasión y visión. Si eres una persona motivada, innovadora y ansiosa por trabajar en un entorno de apoyo, considera explorar nuestras oportunidades laborales en Hugging Face, donde tus contribuciones pueden tener un impacto real.

Juntos, desbloqueemos el potencial de la IA, haciéndola un aliado poderoso para todos.

## 
## Carreras en Hugging Face

En Hugging Face, creemos que el futuro de la IA se trata de hacer que la tecnología sea accesible y beneficiosa para todos. Estamos en una misión para democratizar el aprendizaje automático y transformar la forma en que construimos, compartimos y nos comunicamos con modelos de IA. Si te apasiona la IA, la colaboración y la innovación, ¡puedes encontrar tu próximo camino con nosotros!

### ¿Por qué trabajar con nosotros?

**Cultura Inclusiva y Colaborativa**  
En Hugging Face, nuestra cultura se basa en el trabajo en equipo y la inclusividad. Entendemos que las perspectivas diversas alimentan la innovación, así que invitamos a personas de todos los ámbitos de la vida a unirse a nuestro equipo. Fomentamos un entorno amigable y abierto donde cada voz es valorada y todos pueden contribuir con sus ideas, independientemente de su formación.

**Equilibrio entre Trabajo y Vida Personal**  
Priorizamos el bienestar de nuestros miembros del equipo. Con horarios de trabajo flexibles y la opción de trabajar de forma remota, promovemos un saludable equilibrio entre el trabajo y la vida personal que te permite destacar tanto profesional como personalmente. Creemos que un equipo feliz es un equipo productivo.

**Aprendizaje y Crecimiento Continuos**  
Unirse a Hugging Face significa sumergirse en un mundo de aprendizaje continuo. Trabajarás junto a expertos líderes en IA y aprendizaje automático, mejorarás tus habilidades y desarrollarás tu carrera en un entorno de apoyo y desafío. Ofrecemos oportunidades de desarrollo profesional, mentoría y proyectos prácticos que marcan la diferencia.

### Oportunidades a la Vista

Estamos constantemente buscando personas talentosas para unirse a nuestro diverso equipo. Ya seas un ingeniero experimentado, un investigador en ciernes o un diseñador brillante, hay un lugar para ti en Hugging Face. Actualmente, nuestras ofertas de trabajo incluyen:

- **Ingenieros de Aprendizaje Automático**: Ayúdanos a avanzar en las capacidades de nuestros modelos de código abierto y a llevar tecnología de vanguardia a un público más amplio.
- **Científicos de Datos**: Trabaja con grandes conjuntos de datos para extraer información que mejore nuestros productos y fomente la innovación.
- **Gerentes de Producto**: Guía la hoja de ruta de nuestros productos de IA y asegúrate de que cumplamos con las altas expectativas de nuestra comunidad.
- **Gerentes de Comunidad**: Interactúa con nuestra apasionada base de usuarios y promueve la colaboración dentro de la comunidad de IA.
- **Desarrolladores de Software**: Contribuye a la robustez y escalabilidad de nuestra plataforma, asegurando una experiencia fluida para los usuarios.

### ¡Únete a la Familia Hugging Face!

Si compartes nuestra visión de crear un panorama de IA más inclusivo e innovador, considera ser parte de nuestro equipo. Consulta nuestra [página de carreras](https://huggingface.co/careers) para ver las últimas ofertas de trabajo y aprender más sobre el proceso de aplicación. En Hugging Face, no solo estamos construyendo IA; estamos construyendo una comunidad. ¡Ven y sé parte de ella!

## 
# Información de Contacto

Para cualquier consulta, comentario o soporte, puedes ponerte en contacto con Hugging Face a través de los siguientes canales:

### Consultas Generales
- **Correo Electrónico:** info@huggingface.co
- **Sitio Web:** [Hugging Face](https://huggingface.com)

### Soporte
Para soporte técnico o asistencia:
- **Página de Soporte:** [Hugging Face Support](https://huggingface.co/support)

### Redes Sociales
Mantente conectado con nosotros a través de las redes sociales:
- **Twitter:** [@HuggingFace](https://twitter.com/huggingface)
- **LinkedIn:** [Hugging Face LinkedIn](https://www.linkedin.com/company/hugging-face)

### Dirección
Hugging Face  
[123 Avenida, Ciudad, Estado, Código Postal]  
*Nota: Por favor verifica la dirección exacta a través del sitio web, ya que puede cambiar.*

---

¡No dudes en ponerte en contacto si tienes alguna pregunta o necesitas más información!